# Vectors, Vectors, Vectors

As with many things in life, it all boils down to linear algebra and a few non-linear functions.

Vector representations enables similarity calculations and we can think of several applications that follow from it: question answering, evaluation procedures, fetching related texts, etc. Because of this usefulness, we want to find efficient ways of 1) obtaining vector representations, 2) operating on them, and 3) storing them for later use.

In this notebook, we will discuss how to obtain embeddings from OpenAI API and use a vector database to store and operate on vector representations.

In [ ]:
%load_ext dotenv
%dotenv ../../05_src/.secrets

In [ ]:
import os
from openai import OpenAI
client = OpenAI()

Our sample phrases cover three topics: freedom, friendship, and food.

In [ ]:
phrases = [
    # Freedom
    "Freedom consists not in doing what we like, but in having the right to do what we ought.",
    "Those who deny freedom to others deserve it not for themselves.",
    "Liberty, when it begins to take root, is a plant of rapid growth.",
    "Freedom lies in being bold.",
    "Is freedom anything else than the right to live as we wish?",
    "I am no bird and no net ensnares me: I am a free human being with an independent will.",
    "The secret to happiness is freedom... And the secret to freedom is courage."
    "Freedom is the oxygen of the soul.", 
    "Life without liberty is like a body without spirit."
    # Friendship
    "There is nothing on this earth more to be prized than true friendship.",
    "There are no strangers here; Only friends you haven’t yet met.",
    "Friendship is the only cement that will ever hold the world together.",
    "A true friend is someone who is there for you when he'd rather be anywhere else.",
    "Friendship is the golden thread that ties the heart of all the world.", 
    "Your friend is the man who knows all about you and still likes you.",
    "A single rose can be my garden... a single friend, my world."
    # Food
    "One cannot think well, love well, sleep well, if one has not dined well.",
    "Let food be thy medicine and medicine be thy food.",
    "People who love to eat are always the best people.",
    "The only way to get rid of a temptation is to yield to it.",
    "Food is our common ground, a universal experience.",
    "Life is uncertain. Eat dessert first.",
    "All you need is love. But a little chocolate now and then doesn't hurt."
]

We have 14 phrases in total:

In [ ]:
len(phrases)

To obtain embeddings, we will use the `text-embedding-3-small` model. This model generates 1536-dimensional vectors for each input text. 

The documentation for the embeddings API can be found [here](https://platform.openai.com/docs/guides/embeddings).

# A Simple Input

We first start with a simple example using the first document/phrase:

In [ ]:
phrases[0]

In [ ]:
client = OpenAI()
response = client.embeddings.create(
    input = phrases[0], 
    model = "text-embedding-3-small"
)

In [ ]:
response.data

# Loop through Simple Inputs

We will now try the example found in the [API documentation](https://platform.openai.com/docs/guides/embeddings/embeddings#obtaining-the-embeddings), which simply loops through the documents, calling the API each time. The function below first performs a simple cleanup (removes line breaks), then requests the embeddings.

In [ ]:
def get_embedding(text, model="text-embedding-3-small"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

Using Python's list comprehension syntax, we can run the function for each of our example phrases.

In [ ]:
embeddings = [get_embedding(doc) for doc in phrases]


The statement above is roughly equivalent to:

In [ ]:
embeddings = []
for doc in phrases:
    doc_emb = get_embedding(doc)
    embeddings.append(doc_emb)
embeddings

# Sending Lists of Inputs to the API

We can also send a collection of inputs to the API:

In [ ]:
client = OpenAI()
response = client.embeddings.create(
    input = phrases, 
    model = "text-embedding-3-small"
)
response.data

# Vector DB

We can use a specialized database to store our embeddings, relate them to documents, and efficiently perform computations like cosine similarity.

![](img/02_chroma.png)

The document database that we will use for our experiments is Chroma DB, a simple implementation of Vector DB that is commonly used for prototyping. 

A few useful references are: 
- [ChromaDB Documentation](https://docs.trychroma.com/docs/overview/introduction).
- [ChromaDB Cookbook](https://cookbook.chromadb.dev/running/running-chroma/#chroma-cli).

Chroma can be run locally in memory, locally using file persistence, or using a Docker container.

## Running Chroma Locally in Memory

The simplest implementation is to run Chroma DB in memory without persistence.

In [ ]:
import chromadb

chroma_client = chromadb.Client()

First, create a collection. A collection is a container that groups documents together. A collection would be equivalent to a table which groups togher records in a relational database.

In [ ]:
collection = chroma_client.create_collection(name = "nice_phrases")

Then, add documents to our collection. Each document will contain:

1. An identifier.
2. The phrase.
3. The embeddings.

In [ ]:
embeddings = [item.embedding for item in response.data]
ids = [f"id{i}" for i in range(len(phrases))]

In [ ]:
collection.add(embeddings = embeddings, 
               documents = phrases, 
               ids = ids)

Now, we can use Chroma DB's [`query`](https://docs.trychroma.com/docs/querying-collections/query-and-get) method to perform a query using similarity search. 

## Performing a Search Using Custom Embeddings

We could use a function such as the one below to provide our own embeddings of the query text.

In [ ]:
def query_chromadb(query, top_n = 2):
    query_embedding = get_embedding(query)
    results = collection.query(query_embeddings = [query_embedding], n_results = top_n)
    return [(id, score, text) for id, score, text in zip(results['ids'][0], results['distances'][0], results['documents'][0])]

In [ ]:
query = "What is good food?"

query_chromadb(query, top_n=3)

## Performing a Search Using Embedding Function

Alternatively, we can define the embedding function at the moment in which we create the collection.

If needed, list and remove any collection as you require:

In [ ]:
chroma_client.list_collections()

In [ ]:
chroma_client.delete_collection("nice_phrases")

We can now re-use the collection name using an OpenAI embedding function. Notice that we pass the `api_key` parameter explicitly, as the environment variable name that holds the API key for Chroma DB and for the OpenAI library are different.

In [ ]:
import os
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

collection = chroma_client.create_collection(
    name = "nice_phrases",
    embedding_function = OpenAIEmbeddingFunction(
        api_key = os.getenv("OPENAI_API_KEY"),
        model_name="text-embedding-3-small")
)
collection.add(embeddings = embeddings, 
               documents = phrases, 
               ids = ids)

With the embedding function, we can now perform the query:

In [ ]:
collection.query(
    query_texts = ["What is a friend?", "What is good food?"], 
    n_results = 2
)